In [1]:
import pandas as pd

import nltk

from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline #make_pipeline, make_union

from sklearn.model_selection import train_test_split
# 

from collections import Counter

In [2]:
df_instances = pd.read_json("../data/instances.jsonl", lines=True);
df_truth = pd.read_json("../data/truth.jsonl", lines=True); 

# Display first 5 rows of both
display(df_instances.head())
display(df_truth.head())

ValueError: Expected object or value